In [1]:
#提取数据
import numpy as np  # 用于数学计算和数组操作
import pandas as pd  # 用于数据处理和分析
import matplotlib.pyplot as plt  # 用于数据可视化
import polars as pl  # 高性能数据处理库，类似于pandas
import datetime  # 用于处理日期和时间
from tqdm import tqdm  # 用于显示进度条
import plotly.express as px  # 用于交互式数据可视化
from plotly.subplots import make_subplots  # 用于创建子图
import plotly.graph_objects as go  # 用于创建更复杂的图表
from sklearn.model_selection import train_test_split  # 用于划分训练集和测试集
import sys
sys.path.append('/home/zhuangzhuohan/sleep_data')
from my_py.my_funs import make_train_dataset_new,load_and_preprocess_data_new,create_features_new,apply_features,analyze_validation_data,analyze_val_data_stats,create_features_float,load_and_preprocess_data_float
tolerances = {
    'onset': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],  # 睡眠开始事件的时间容忍度（分钟）
    'wakeup': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360]  # 睡眠结束事件的时间容忍度（分钟）
}
# 1. 加载和预处理数据
train_series, train_events, test_series ,series_ids = load_and_preprocess_data_float()
# 2. 创建特征
features, feature_cols, id_cols = create_features_float()
# 3. 应用特征到训练数据
train_series = apply_features(train_series, features, id_cols, feature_cols)
test_series = apply_features(test_series, features, id_cols, feature_cols)
# 2. 定义列名映射（用于评分函数）
column_names = {
    'series_id_column_name': 'series_id',
    'time_column_name': 'step',
    'event_column_name': 'event',
    'score_column_name': 'score',
}
# 3. 划分训练集和验证集（70%训练，30%验证）
train_ratio = 0.7
random_seed = 42
train_ids, val_ids = train_test_split(series_ids, train_size=train_ratio, random_state=random_seed)
# 4. 收集训练数据
train_data = train_series.filter(pl.col('series_id').is_in(train_ids)).collect()
# 转换为pandas DataFrame后使用切片方法
step_time = 12  # 每1分钟取一个数据点
train_data = train_data.to_pandas().iloc[::(step_time)]  # 每step_time分钟取一个数据点
train_data = pl.from_pandas(train_data)  # 转回polars DataFrame
# 创建训练事件数据（只包含训练集的事件）
train_solution_series_id = train_events.filter(pl.col('series_id').is_in(train_ids))
train_solution = train_events.filter(pl.col('series_id').is_in(train_ids)).select(['series_id', 'event', 'step']).to_pandas()
# 统计真实的onset和wakeup事件数量
train_solution_onset_count = len(train_solution[train_solution['event'] == 'onset'])
train_solution_wakeup_count = len(train_solution[train_solution['event'] == 'wakeup'])
print(f"train_solution的onset事件数量: {train_solution_onset_count}")
print(f"train_solution的wakeup事件数量: {train_solution_wakeup_count}")
# 创建训练数据集
X_train, y_train = make_train_dataset_new(train_data, train_solution_series_id, feature_cols, id_cols)
# 5. 创建验证数据
val_data = train_series.filter(pl.col('series_id').is_in(val_ids)).collect()
# 6. 创建验证标签（用于评估模型性能）
val_solution = train_events.filter(pl.col('series_id').is_in(val_ids)).select(['series_id', 'event', 'step']).to_pandas()

train_solution的onset事件数量: 3416
train_solution的wakeup事件数量: 3416


100%|██████████| 188/188 [00:00<00:00, 191.01it/s]


In [12]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import os
# 定义模型超参数字典
model_params = {
    'n_estimators': 10,              # 决策树数量（默认：100）
    'criterion': 'gini',             # 分裂标准（默认：'gini'）
    'max_depth': 20,                 # 树的最大深度（默认：None）
    'min_samples_split': 2,          # 分裂所需的最小样本数（默认：2）
    'min_samples_leaf': 100,         # 每个叶节点最少样本数（默认：1）
    'min_weight_fraction_leaf': 0.0, # 叶节点最小权重分数（默认：0.0）
    'max_features': 'sqrt',          # 寻找最佳分裂时考虑的特征数量（默认：'sqrt'）
    'max_leaf_nodes': None,          # 最大叶节点数（默认：None）
    'min_impurity_decrease': 0.0,    # 最小不纯度减少（默认：0.0）
    'bootstrap': True,               # 是否使用bootstrap抽样（默认：True）
    'oob_score': False,              # 是否使用袋外样本评估（默认：False）
    'n_jobs': -1,                    # 使用所有CPU核心（默认：None）
    'random_state': random_seed,     # 随机种子（默认：None）
    'verbose': 0,                    # 详细程度（默认：0）
    'warm_start': False,             # 是否使用上次的拟合结果作为起点（默认：False）
    'class_weight':'balanced',     # 类别权重（默认：None）
    'ccp_alpha': 0.0,                # 剪枝参数（默认：0.0）
    'max_samples': None,             # bootstrap抽样的最大样本数（默认：None）
    'monotonic_cst': None            # 单调约束（默认：None）
}
# 初始化随机森林分类器
rf_classifier = RandomForestClassifier(random_state=model_params['random_state'])
# 训练分类器（设置超参数）
rf_classifier = RandomForestClassifier(**model_params)
# 拟合模型
rf_classifier.fit(X_train[feature_cols], y_train)
# 获取特征重要性
feature_importances = rf_classifier.feature_importances_
total_importance = sum(feature_importances)
# 准备数据
importance_data = []
print("特征重要性占比:")
for feature, importance in sorted(zip(feature_cols, feature_importances), 
                                 key=lambda x: x[1], reverse=True):
    importance_ratio = (importance / total_importance) * 100
    print(f"{feature:25} | 重要性: {importance:.6f} | 占比: {importance_ratio:.2f}%")
    importance_data.append({
        'feature': feature,
        'importance': importance,
        'importance_ratio': importance_ratio
    })
print(f"总重要性: {total_importance:.6f}")
# 创建数据框
importance_df = pd.DataFrame(importance_data)
# 保存到CSV文件（追加模式）
csv_file = 'importance.csv'
if os.path.exists(csv_file):
    # 文件存在，追加数据
    importance_df.to_csv(csv_file, mode='a', header=False, index=False)
    print(f"特征重要性已追加到 {csv_file}")
else:
    # 文件不存在，创建新文件
    importance_df.to_csv(csv_file, index=False)
    print(f"特征重要性已保存到 {csv_file}")

特征重要性占比:
anglez_1v_1m_mean         | 重要性: 0.127885 | 占比: 12.79%
enmo_1v_30m_mean          | 重要性: 0.126713 | 占比: 12.67%
enmo_1v_15m_mean          | 重要性: 0.124399 | 占比: 12.44%
enmo_1v_60m_mean          | 重要性: 0.065882 | 占比: 6.59%
enmo_1v_10m_mean          | 重要性: 0.060973 | 占比: 6.10%
anglez_1v_1m_max          | 重要性: 0.059063 | 占比: 5.91%
enmo_1v_1m_mean           | 重要性: 0.058010 | 占比: 5.80%
anglez_1v_240m_max        | 重要性: 0.046219 | 占比: 4.62%
anglez_1v_240m_std        | 重要性: 0.033767 | 占比: 3.38%
hour                      | 重要性: 0.028366 | 占比: 2.84%
enmo_1v_120m_mean         | 重要性: 0.027719 | 占比: 2.77%
anglez_1v_60m_max         | 重要性: 0.020520 | 占比: 2.05%
anglez_1v_120m_max        | 重要性: 0.019398 | 占比: 1.94%
anglez_1v_60m_mean        | 重要性: 0.017985 | 占比: 1.80%
anglez_1v_120m_mean       | 重要性: 0.017026 | 占比: 1.70%
enmo_1v_240m_max          | 重要性: 0.016742 | 占比: 1.67%
anglez_1m_std             | 重要性: 0.015070 | 占比: 1.51%
anglez_240m_max           | 重要性: 0.014621 | 占比: 1.46%
anglez_2m_std   

In [13]:
from my_py.metric import score  # 导入自定义的评分函数
from my_py.model_analyzer import analyze_sleep_model_performance
from my_py.my_funs import get_events_new
# 在验证集上检查模型性能
min_sleep_duration = 12*30
rf_submission = get_events_new(val_data, rf_classifier, feature_cols, id_cols, min_sleep_duration)  # 生成验证集的预测事件
# 调用函数，传入 rf_classifier 参数以计算内存
performance_results = analyze_sleep_model_performance(
    rf_submission=rf_submission,
    ground_truth=val_solution,  # 使用 val_solution 作为真实事件数据
    val_data=val_data,
    val_ids=val_ids,
    rf_classifier=rf_classifier,  # 传入训练好的分类器模型
    min_sleep_duration=min_sleep_duration  # 可根据需要调整最小睡眠周期长度
)
# 计算模型得分
rf_score = score(val_solution, rf_submission, tolerances, **column_names)
print(f"Random forest score: {rf_score}")

100%|██████████| 81/81 [00:06<00:00, 12.64it/s]



模型内存占用: 6.27 MB
验证数据总天数: 2294
预测天数: 1724
预测率: 0.7515
onset事件率: 0.9969
wakeup事件率: 1.4965
预测onset事件数: 2287
预测wakeup事件数: 3433
真实onset事件数: 1374
真实wakeup事件数: 1374
Onset匹配数(5/10/15/30/60/120)分钟：773/985/1130/1248/1314/1355
Onset匹配率(5/10/15/30/60/120)分钟：0.5626/0.7169/0.8224/0.9083/0.9563/0.9862
Onset平均差(秒)(5/10/15/30/60/120)分钟：155.92/216.35/283.54/378.83/487.88/708.21
Onset标准差(秒)(5/10/15/30/60/120)分钟：71.93/138.35/219.77/369.97/596.09/1179.23

Wakeup匹配数(5/10/15/30/60/120)分钟：322/626/788/1068/1255/1336
Wakeup匹配率(5/10/15/30/60/120)分钟：0.2344/0.4556/0.5735/0.7773/0.9134/0.9723
Wakeup平均差(秒)(5/10/15/30/60/120)分钟：199.39/313.07/401.52/636.47/904.44/1155.83
Wakeup标准差(秒)(5/10/15/30/60/120)分钟：69.37/138.94/217.07/456.41/784.04/1260.59

总和匹配数(5/10/15/30/60/120)分钟：1095/1611/1918/2316/2569/2691
总和匹配率(5/10/15/30/60/120)分钟：0.3985/0.5862/0.6980/0.8428/0.9349/0.9793
总和平均差(秒)(5/10/15/30/60/120)分钟：168.70/253.93/332.02/497.64/691.38/930.44
总和标准差(秒)(5/10/15/30/60/120)分钟：73.89/146.38/226.24/431.64/724.84/1240.65

Rand

/home/zhuangzhuohan/sleep_data/my_py/metric.py:433: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .groupby([event_column_name, 'tolerance'], group_keys=False).apply(
